In [3]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams["figure.dpi"]=300
import torch
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr,spearmanr

from src.utils import reconst_pair_idx,create_total_df

def discritize(logit,thresh=0.5):
    pred = 1 if logit>=thresh else 0
    return pred


seq_df = pd.read_csv('/home/ksuga/whole_mrna_predictor/UTR_PairPred/data/human/gencode44_utr_gene_unique_cdhit09.csv')
remover= lambda x:x.split(".")[0]

#cl_dir = "/home/ksuga/whole_mrna_predictor/UTR_PairPred/results/runs/contrastive_learning_10fold_shuffle_seed0"
cl_dir = "/home/ksuga/whole_mrna_predictor/UTR_PairPred/results/runs/contrastive_learning_10fold_rinalmo_whole_ave_seed1"
tot_df_cl = create_total_df(cl_dir,seq_df)
tot_df_cl=tot_df_cl.drop("ENST_ID",axis=1).rename(columns={"ENST_ID_PRE":"ENST_ID"})

In [5]:
tot_df_cl

,utr5,utr3,label,pred,correct,logits,cos_sim,ENST_ID,GENE,3UTR,total_len,5UTR_len,CDS_len,3UTR_len
293,9257,9257,1,True,True,0.705522,0.873735,ENST00000528841,BRSK2,ACACACUAGCCCCCCCCCCCAGCACAGCACUGACAGCGGCUGCCUC...,4528,351,2211,1966
296,3716,3716,1,True,True,0.696336,0.829912,ENST00000618052,H3C7,GUUGUACUGAGGGUGUGCGCCAACUUAAACCAAAGGCUCUUUUCAG...,494,30,411,53
299,3520,3520,1,True,True,0.695916,0.827926,ENST00000328345,POU3F2,ACUCGAGCUGGGGGAGGGGCAGAGCGCGGGGCUCCCCCUCCCCUUC...,4885,300,1332,3253
405,1384,1384,1,True,True,0.694372,0.820638,ENST00000293441,SHANK1,UGGCUGGCCUGGACGGACCAGCCCCGUCCACAGAACUCUUGAGCCU...,9636,440,6486,2710
28,7625,7625,1,True,True,0.694133,0.819514,ENST00000394166,NR2F2,AUAAAUAAAAUAAGAAGGGGGAGUGAAACAGAGAAAGAAAAGGCAA...,5287,1406,1245,2636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,3689,3689,1,False,False,0.396502,-0.420062,ENST00000567435,UBL7,ACUCCCUGCUUCCCCUGAACCCCCAGCAAGUUGCAGAGGCUACUGC...,1720,464,1143,113
784,6853,6853,1,False,False,0.395291,-0.425126,ENST00000306324,HOXD4,AAGUGGGGACCCUGGGCCCAUCUCUCCCUGCGCACCAGGCUGAGCC...,1136,84,768,284
1622,7255,7255,1,False,False,0.394273,-0.429387,ENST00000337014,HEXD,UCUGCAUGUGUGACACUGAUUCUUUGGAAAUAAAGAGUGGAAGCUG...,2285,474,1758,53
458,9617,9617,1,False,False,0.392457,-0.436998,ENST00000334134,FGF3,CUGGGCCUGGUGGCCACCGCCAGAGCUCCUGGCGACAUCUUGGCGU...,1540,483,720,337


In [24]:
tot_df_cl

,utr5,utr3,label,pred,correct,logits,cos_sim,ENST_ID,GENE,3UTR,total_len,5UTR_len,CDS_len,3UTR_len
293,9257,9257,1,True,True,0.705522,0.873735,ENST00000528841,BRSK2,ACACACUAGCCCCCCCCCCCAGCACAGCACUGACAGCGGCUGCCUC...,4528,351,2211,1966
296,3716,3716,1,True,True,0.696336,0.829912,ENST00000618052,H3C7,GUUGUACUGAGGGUGUGCGCCAACUUAAACCAAAGGCUCUUUUCAG...,494,30,411,53
299,3520,3520,1,True,True,0.695916,0.827926,ENST00000328345,POU3F2,ACUCGAGCUGGGGGAGGGGCAGAGCGCGGGGCUCCCCCUCCCCUUC...,4885,300,1332,3253
405,1384,1384,1,True,True,0.694372,0.820638,ENST00000293441,SHANK1,UGGCUGGCCUGGACGGACCAGCCCCGUCCACAGAACUCUUGAGCCU...,9636,440,6486,2710
28,7625,7625,1,True,True,0.694133,0.819514,ENST00000394166,NR2F2,AUAAAUAAAAUAAGAAGGGGGAGUGAAACAGAGAAAGAAAAGGCAA...,5287,1406,1245,2636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,3689,3689,1,False,False,0.396502,-0.420062,ENST00000567435,UBL7,ACUCCCUGCUUCCCCUGAACCCCCAGCAAGUUGCAGAGGCUACUGC...,1720,464,1143,113
784,6853,6853,1,False,False,0.395291,-0.425126,ENST00000306324,HOXD4,AAGUGGGGACCCUGGGCCCAUCUCUCCCUGCGCACCAGGCUGAGCC...,1136,84,768,284
1622,7255,7255,1,False,False,0.394273,-0.429387,ENST00000337014,HEXD,UCUGCAUGUGUGACACUGAUUCUUUGGAAAUAAAGAGUGGAAGCUG...,2285,474,1758,53
458,9617,9617,1,False,False,0.392457,-0.436998,ENST00000334134,FGF3,CUGGGCCUGGUGGCCACCGCCAGAGCUCCUGGCGACAUCUUGGCGU...,1540,483,720,337


In [23]:
low_thresh=-1.0
high_thresh=1.0
target_df = tot_df_cl[(tot_df_cl['cos_sim']>low_thresh)&(tot_df_cl['cos_sim']<high_thresh)]
utr3 = target_df['3UTR'].values
tot_sample=len(utr3)
target_motifs = {1:'AUUUA',2:'UUAUUUAUU',3:'UUAUUUAUA',4:'UUAUUUAAU',5:'UUAUUUAAA'}
motif_counts = {i+1:0 for i in range(len(target_motifs))}
for seq in utr3:
    for num,motif in target_motifs.items():
        if motif in seq:
            motif_counts[num]=motif_counts[num]+1
motif_rate = {i:f"{(num/tot_sample):.3f}" for i,num in motif_counts.items()}
print(motif_counts)
print(motif_rate)

{1: 12068, 2: 1522, 3: 873, 4: 837, 5: 1263}
{1: '0.718', 2: '0.090', 3: '0.052', 4: '0.050', 5: '0.075'}


In [2]:
## For contrastive learning
data_dir=f"./results/runs/contrastive_learning_seed5"
score_path=os.path.join(data_dir,"score_dict.pkl")
preds_path=os.path.join(data_dir,"pred_results.pkl")

with open(score_path,"rb") as f:
    score=pickle.load(f)

with open(preds_path,"rb") as f:
    pred_results=pickle.load(f) # (out_cos_sim,out_logits,pair_idx_list)

print(score)


{'accuracy': 0.6580357142857143, 'precision': 0.6137430644472898, 'recall': 0.8549346016646849, 'f1': 0.7145341614906832, 'confusion_matrix': array([[ 773,  905],
       [ 244, 1438]]), 'fpr_all': array([0.        , 0.        , 0.        , ..., 0.9988081 , 0.99940405,
       1.        ]), 'tpr_all': array([0.00000000e+00, 5.94530321e-04, 1.18906064e-03, ...,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00]), 'roc_thresh': array([1.6996603 , 0.69966036, 0.6982985 , ..., 0.36262056, 0.36194405,
       0.35727412], dtype=float32), 'auc_roc': 0.728652889247292}


In [4]:
seq_df = pd.read_csv("./data/human/gencode44_utr_gene_unique_cdhit09.csv",index_col=0)
pair_idx = reconst_pair_idx(pred_results[-1])
logits=pred_results[1].reshape(-1)
cos_sim = pred_results[0].reshape(-1)

In [7]:
df_pred_res=pd.DataFrame(pair_idx,columns=["utr5","utr3","label"])
df_pred_res["pred"] = list(map(discritize,pred_results[1]))
df_pred_res["correct"] = (df_pred_res["label"]==df_pred_res["pred"]).values
df_pred_res["logits"] = logits
df_pred_res["cos_sim"] = cos_sim
df_pred_res["ENST_ID"]=seq_df.iloc[df_pred_res.utr5.values]["ENST_ID"].values
df_pred_res["ENST_ID_PRE"]=list(map(lambda enst_id: enst_id.split(".")[0],df_pred_res["ENST_ID"].values))
df_pred_res["GENE"]=seq_df.iloc[df_pred_res.utr5.values]["GENE"].values
df_pred_res=pd.concat([df_pred_res,seq_df.iloc[:,-4:].iloc[df_pred_res.utr5.values].reset_index(drop=True)],axis=1)
df_pred_res.sort_values("cos_sim",ascending=False,inplace=True)
#df_pred_res.to_csv(f"human_pred_analysis_seed{seed}.csv")

df_pred_res = df_pred_res[(df_pred_res['label']==1)]
df_pred_res

NameError: name 'pair_idx' is not defined

In [18]:
## For multi set. If you want to concat results from 5 seeds
flg=1
for i in range(5):
    dir  =f"./results/runs/contrastive_learning_seed{i+1}"
    pred_path = os.path.join(dir,"pred_results.pkl")
    with open(pred_path,"rb") as f:
        pred_results=pickle.load(f) # (out_cos_sim,out_logits,pair_idx_list)
        pair_idx = reconst_pair_idx(pred_results[-1])
        logits=pred_results[1].reshape(-1)
        cos_sim = pred_results[0].reshape(-1)

    df_pred_res=pd.DataFrame(pair_idx,columns=["utr5","utr3","label"])
    df_pred_res["pred"] = list(map(discritize,pred_results[1]))
    df_pred_res["correct"] = (df_pred_res["label"]==df_pred_res["pred"]).values
    df_pred_res["logits"] = logits
    df_pred_res["cos_sim"] = cos_sim
    df_pred_res["ENST_ID"]=seq_df.iloc[df_pred_res.utr5.values]["ENST_ID"].values
    df_pred_res["ENST_ID_PRE"]=list(map(lambda enst_id: enst_id.split(".")[0],df_pred_res["ENST_ID"].values))
    df_pred_res["GENE"]=seq_df.iloc[df_pred_res.utr5.values]["GENE"].values
    df_pred_res["5UTR"]=seq_df.iloc[df_pred_res.utr5.values]["5UTR"].values
    df_pred_res["3UTR"]=seq_df.iloc[df_pred_res.utr5.values]["3UTR"].values
    df_pred_res=pd.concat([df_pred_res,seq_df.iloc[:,-4:].iloc[df_pred_res.utr5.values].reset_index(drop=True)],axis=1)
    df_pred_res.sort_values("cos_sim",ascending=False,inplace=True)
    if flg:
        total_df = df_pred_res
        flg=0
    else:
        total_df = pd.concat([total_df,df_pred_res])
    
    total_df.sort_values("cos_sim",ascending=False,inplace=True)
    total_df = total_df[(total_df["label"]==1)]
target_df=total_df[~total_df.duplicated(subset="ENST_ID_PRE")]


In [2]:
import pickle
with open("./data/human/go_enst_table.pkl","rb") as f:
    go_enst_table,enst_go_table = pickle.load(f)

brain_go_id = ["GO:0007420"]
target_enst_id = []
for go_id in brain_go_id:
    target_enst_id.extend(go_enst_table[go_id])
print(len(target_enst_id))

476


In [11]:
target_df_filt = target_df.iloc[:200]
top_go_ids =[]
for enst_id in  target_df_filt["ENST_ID_PRE"].values:
    gos = enst_go_table.get(enst_id)
    if gos==None:
        continue
    else:
        top_go_ids.extend(enst_go_table[enst_id])

In [12]:
u,c = np.unique(top_go_ids,return_counts=True)
go_df = pd.DataFrame({"GO":u,"count":c})
go_df.sort_values("count",ascending=False)

,GO,count
217,GO:0005515,119
230,GO:0005634,76
262,GO:0005886,61
231,GO:0005654,47
554,GO:0016020,44
...,...,...
698,GO:0030168,1
696,GO:0030163,1
695,GO:0030160,1
690,GO:0030139,1


In [66]:
match = list(set(target_df_filt["ENST_ID_PRE"].values)&(set(target_enst_id)))
matched_df = target_df_filt.set_index("ENST_ID_PRE").loc[match]
print(matched_df.shape)
matched_df["cos_sim"].mean()

(3, 13)


0.8548749

## Create fasta for STREME motif analysis

In [26]:
tmp_df = target_df[(target_df["correct"]==1)&(target_df["cos_sim"]>0.8)]

In [3]:
with open('./data/human/CL_CosSim_bin08_5utr.fa',"w") as f5, open('./data/human/CL_CosSim_bin08_3utr.fa',"w") as f3 : 
    for enst_id,utr5,utr3 in tmp_df[['ENST_ID_PRE','5UTR','3UTR']].values:
        sequence_id = f">{enst_id}"
        f5.write(f"{sequence_id}\n{utr5}\n")
        f3.write(f"{sequence_id}\n{utr3}\n")

    

NameError: name 'tmp_df' is not defined

In [ ]:
import pandas as pd

# FASTAファイルに書き込む関数
def write_fasta_file(df, filename):
    with open(filename, 'w') as f:
        for index, row in df.iterrows():
            sequence_id = f">{row['ID']}"
            sequence = row['Sequence']
            f.write(f"{sequence_id}\n{sequence}\n")

# FASTAファイルに書き込む
write_fasta_file(df, 'output.fasta')
